# Eval

In [ ]:
from random import seed, shuffle

import pandas as pd

from ipymarkup import (
    show_line_markup as show_markup,
    ascii_markup
)

from nerus.log import log_progress
from nerus.utils import (
    head,
    iter_sents
)
from nerus.span import select_type_spans
from nerus.load import (
    load_raw,
    load_norm
)
from nerus.path import join_path
from nerus.const import (
    NERUS_,
    DUMPS_DIR, RAW, JSONL, GZ,
    FACTRU, NE5, GAREEV, BSNLP, LENTA,
    DEEPPAVLOV, DEEPPAVLOV_BERT, PULLENTI, TEXTERRA, TOMITA, NATASHA, MITIE, ANNOTATORS,
    PER, LOC, ORG
)
from nerus.eval.score import eval_markups
from nerus.eval.report import (
    report_table,
    format_report,
    format_github_report
)

In [ ]:
def load(name):
    path = join_path(DUMPS_DIR, NERUS_ + name + RAW + JSONL + GZ)
    return list(log_progress(load_raw(path), prefix=name))


factru = load(FACTRU)
gareev = load(GAREEV)
ne5 = load(NE5)
bsnlp = load(BSNLP)

sources = [FACTRU, GAREEV, NE5, BSNLP]
datasets = {
    FACTRU: factru,
    GAREEV: gareev,
    NE5: ne5,
    BSNLP: bsnlp
}

In [ ]:
etalons = {}
for source in sources:
    dataset = datasets[source]
    etalon = [_.source.adapted for _ in log_progress(dataset, prefix=source)]
    etalons[source] = etalon

## Quant

In [ ]:
def eval(source, annotator):
    dataset = datasets[source]
    etalon = etalons[source]
    guess = [_.find(annotator).adapted for _ in dataset]
    return eval_markups(guess, etalon)


annotators = ANNOTATORS
keys = [
    (source, annotator)
    for annotator in annotators
    for source in sources
]
scores = {_: eval(*_) for _ in log_progress(keys)}

In [ ]:
table = report_table(scores, sources, annotators)
output = format_report(table)
output

In [ ]:
# print(output.to_html())

<table border="1" class="dataframe">
  <thead>
    <tr>
      <th></th>
      <th colspan="3" halign="left">factru</th>
      <th colspan="3" halign="left">gareev</th>
      <th colspan="3" halign="left">ne5</th>
      <th colspan="3" halign="left">bsnlp</th>
    </tr>
    <tr>
      <th>prec/recall/f1,%</th>
      <th>PER</th>
      <th>LOC</th>
      <th>ORG</th>
      <th>PER</th>
      <th>LOC</th>
      <th>ORG</th>
      <th>PER</th>
      <th>LOC</th>
      <th>ORG</th>
      <th>PER</th>
      <th>LOC</th>
      <th>ORG</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>deeppavlov</th>
      <td>86/96/90</td>
      <td>85/91/88</td>
      <td>72/75/74</td>
      <td>90/99/94</td>
      <td>-/-/-</td>
      <td>73/86/79</td>
      <td>94/93/94</td>
      <td>86/98/91</td>
      <td>87/88/88</td>
      <td>80/94/86</td>
      <td>63/95/76</td>
      <td>61/63/62</td>
    </tr>
    <tr>
      <th>deeppavlov_bert</th>
      <td>96/97/97</td>
      <td>91/93/92</td>
      <td>85/79/82</td>
      <td>97/99/98</td>
      <td>-/-/-</td>
      <td>88/94/91</td>
      <td>99/99/99</td>
      <td>98/99/99</td>
      <td>98/97/97</td>
      <td>93/97/95</td>
      <td>73/97/84</td>
      <td>79/69/73</td>
    </tr>
    <tr>
      <th>pullenti</th>
      <td>95/85/90</td>
      <td>86/77/81</td>
      <td>68/68/68</td>
      <td>91/96/94</td>
      <td>-/-/-</td>
      <td>63/64/63</td>
      <td>97/92/95</td>
      <td>86/85/86</td>
      <td>62/75/68</td>
      <td>91/88/89</td>
      <td>71/82/76</td>
      <td>58/55/56</td>
    </tr>
    <tr>
      <th>texterra</th>
      <td>91/88/90</td>
      <td>76/83/80</td>
      <td>86/45/59</td>
      <td>87/89/88</td>
      <td>-/-/-</td>
      <td>81/42/56</td>
      <td>97/83/90</td>
      <td>81/73/77</td>
      <td>83/46/59</td>
      <td>86/85/85</td>
      <td>67/92/78</td>
      <td>76/42/54</td>
    </tr>
    <tr>
      <th>tomita</th>
      <td>93/92/92</td>
      <td>-/-/-</td>
      <td>-/-/-</td>
      <td>90/93/92</td>
      <td>-/-/-</td>
      <td>-/-/-</td>
      <td>96/92/94</td>
      <td>-/-/-</td>
      <td>-/-/-</td>
      <td>83/92/88</td>
      <td>-/-/-</td>
      <td>-/-/-</td>
    </tr>
    <tr>
      <th>natasha</th>
      <td>96/78/86</td>
      <td>72/78/75</td>
      <td>41/23/29</td>
      <td>96/79/87</td>
      <td>-/-/-</td>
      <td>40/30/34</td>
      <td>98/75/85</td>
      <td>70/71/70</td>
      <td>49/32/39</td>
      <td>91/76/83</td>
      <td>65/88/75</td>
      <td>45/28/34</td>
    </tr>
    <tr>
      <th>mitie</th>
      <td>95/82/88</td>
      <td>90/81/86</td>
      <td>74/41/53</td>
      <td>85/84/84</td>
      <td>-/-/-</td>
      <td>55/38/45</td>
      <td>94/62/75</td>
      <td>74/56/64</td>
      <td>49/38/43</td>
      <td>81/66/73</td>
      <td>74/86/80</td>
      <td>55/49/52</td>
    </tr>
  </tbody>
</table>

In [ ]:
# output = format_github_report(table)
# html = output.to_html(escape=False)
# html = html.replace('border="1"', 'border="0"')
# print(html)

## Qual

In [ ]:
sents = list(iter_sents(factru))
seed(5)
shuffle(sents)

In [ ]:
# count = 0
# for sent in sents:
#     types = [PER]

#     a = sent.source.adapted
#     a = list(select_type_spans(a.spans, types))

#     b = sent.find(DEEPPAVLOV).adapted
#     b = list(select_type_spans(b.spans, types))

#     if a == b:
#         continue

#     show_markup(sent.text, a)
#     print('---')
#     show_markup(sent.text, b)
#     print('---')

#     for annotator in [TOMITA, PULLENTI, TEXTERRA, MITIE]:
#         markup = sent.find(annotator)
#         print(markup.label)
#         show_markup(sent.text, markup.adapted.spans)
#     print('\n\n\n')

#     count += 1
#     if count > 10:
#         break

## Dump

In [ ]:
path = join_path(DUMPS_DIR, NERUS_ + LENTA + JSONL + GZ)
lenta = load_norm(path)


def format(record):
    markup = ascii_markup(record.text, record.spans)
    for line in markup.as_ascii:
        yield line + '\n'
    yield '\n\n'


with open('examples/lenta_500.txt', 'w') as file:
    for record in head(lenta, 500):
        lines = format(record)
        file.writelines(lines)

In [ ]:
def format(type, source, text, a, b):
    yield '[%s] %s / nerus\n' % (type, source)
    for index, spans in enumerate([a, b]):
        if index > 0:
            yield '>\n'
        markup = ascii_markup(text, spans)
        for line in markup.as_ascii:
            yield line + '\n'
    yield '\n\n'


with open('examples/errors.txt', 'w') as file:
    for records in [factru, gareev, ne5]:
        sents = list(iter_sents(records))
        for type in [PER, LOC, ORG]:
            for sent in sents:
                a = list(select_type_spans(sent.source.adapted.spans, [type]))
                b = list(select_type_spans(sent.find(DEEPPAVLOV_BERT).adapted.spans, [type]))
                if a != b:
                    lines = format(type, sent.source.label, sent.text, a, b)
                    file.writelines(lines)